# Hyundai Ships Crew Estimates Using Linear Regression & PySpark

##Linear Regression on a Hyundai Heavy Industies dataset of ships to estimate crew members needed for a new line of ships created for different customers

In [0]:
# Create a Spark session and import the ships dataset
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('proj').getOrCreate()
data=spark.read.csv('dbfs:/FileStore/shared_uploads/hrishagni95@gmail.com/cruise_ship_info.csv',inferSchema=True,header=True)

In [0]:
# Summarise the dataset
data.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
summary|Ship_name|Cruise_line| Age| Tonnage| passengers| length| cabins|passenger_density| crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
 count| 158| 158| 158| 158| 158| 158| 158| 158| 158|
 mean| Infinity| null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
 stddev| null| null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
 min|Adventure| Azamara| 4| 2.329| 0.66| 2.79| 0.33| 17.7| 0.59|
 max|Zuiderdam| Windstar| 48| 220.0| 54.0| 11.82| 27.0| 71.43| 21.0|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+

In [0]:
# Check the column names of the dataset, so as to know what we are dealing with
data.columns

Out[14]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [0]:
# Check on the data
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 20 rows

In [0]:
# Check the schema of the csv file
data.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)

In [0]:
# Import StringIndexer
from pyspark.ml.feature import StringIndexer

In [0]:
# Use StringIndexer to convert the 'Cruise_line' field into a string indexed field
df = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_Line_Index")
df = indexer.fit(data).transform(data)
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_Line_Index|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7| 1.0|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1| 1.0|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0| 1.0|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2| 1.0|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5| 1.0|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6| 1.0|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3| 1.0|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3| 1.0|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3| 1.0|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
only showing top 20 rows

In [0]:
# Import function LinearRegression and VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
# Create a vector assembler using the below fields to generate the features field, which will be crucial for our linear regression in the next stages
assembler=VectorAssembler(inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density', 
 'Cruise_Line_Index'],outputCol='features')

In [0]:
# Use the vector assembler to transform the indexed dataframe
output_df=assembler.transform(df)

In [0]:
output_df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+--------------------+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_Line_Index| features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+--------------------+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|[6.0,30.276999999...|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|[6.0,30.276999999...|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7| 1.0|[26.0,47.262,14.8...|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1| 1.0|[11.0,110.0,29.74...|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0| 1.0|[17.0,101.353,26....|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[22.0,70.367,20.5...|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[15.0,70.367,20.5...|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2| 1.0|[23.0,70.367,20.5...|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[19.0,70.367,20.5...|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5| 1.0|[6.0,110.23899999...|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|[10.0,110.0,29.74...|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6| 1.0|[28.0,46.052,14.5...|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[18.0,70.367,20.5...|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[17.0,70.367,20.5...|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3| 1.0|[11.0,86.0,21.24,...|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|[8.0,110.0,29.74,...|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3| 1.0|[9.0,88.5,21.24,9...|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[15.0,70.367,20.5...|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3| 1.0|[12.0,88.5,21.24,...|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[20.0,70.367,20.5...|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+--------------------+
only showing top 20 rows

In [0]:
# Select only the necessary fields for the regression model
final_df=output_df.select('crew','features')

In [0]:
final_df.show()

+----+--------------------+
crew| features|
+----+--------------------+
3.55|[6.0,30.276999999...|
3.55|[6.0,30.276999999...|
 6.7|[26.0,47.262,14.8...|
19.1|[11.0,110.0,29.74...|
10.0|[17.0,101.353,26....|
 9.2|[22.0,70.367,20.5...|
 9.2|[15.0,70.367,20.5...|
 9.2|[23.0,70.367,20.5...|
 9.2|[19.0,70.367,20.5...|
11.5|[6.0,110.23899999...|
11.6|[10.0,110.0,29.74...|
 6.6|[28.0,46.052,14.5...|
 9.2|[18.0,70.367,20.5...|
 9.2|[17.0,70.367,20.5...|
 9.3|[11.0,86.0,21.24,...|
11.6|[8.0,110.0,29.74,...|
10.3|[9.0,88.5,21.24,9...|
 9.2|[15.0,70.367,20.5...|
 9.3|[12.0,88.5,21.24,...|
 9.2|[20.0,70.367,20.5...|
+----+--------------------+
only showing top 20 rows

In [0]:
# Use randomsplit() method to split the dataframe in to training data set and testing data set. Use a 70/30 split ratio.
train_data,test_data=final_df.randomSplit([0.7,0.3])

In [0]:
# Create the LinearRegression model
lr=LinearRegression(labelCol='crew')

In [0]:
# Use the regression model to fit the training data
lr_model=lr.fit(train_data)

In [0]:
# After the training data is fitted on the regression model, use the fit to evaluate the testing data set
res=lr_model.evaluate(test_data)

In [0]:
# Check on the R2 score to understand how well our model has been trained to correctly predict the crew estimates
res.r2

Out[32]: 0.8828322301148132

In [0]:
# A R2 score of 0.88 means that we have been quite succesful in our modelling so far

In [0]:
# After model training and testing, we will now check the actual crew estimate that Hyundai is looking for by transforming a unlabeled data set, where the 'crew' field does not exist
unlabeled_data=output_df.select('features')
unlabeled_data.show()

+--------------------+
 features|
+--------------------+
[6.0,30.276999999...|
[6.0,30.276999999...|
[26.0,47.262,14.8...|
[11.0,110.0,29.74...|
[17.0,101.353,26....|
[22.0,70.367,20.5...|
[15.0,70.367,20.5...|
[23.0,70.367,20.5...|
[19.0,70.367,20.5...|
[6.0,110.23899999...|
[10.0,110.0,29.74...|
[28.0,46.052,14.5...|
[18.0,70.367,20.5...|
[17.0,70.367,20.5...|
[11.0,86.0,21.24,...|
[8.0,110.0,29.74,...|
[9.0,88.5,21.24,9...|
[15.0,70.367,20.5...|
[12.0,88.5,21.24,...|
[20.0,70.367,20.5...|
+--------------------+
only showing top 20 rows

In [0]:
pred=lr_model.transform(unlabeled_data)
pred.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[6.0,30.276999999...| 4.245375692178367|
[6.0,30.276999999...| 4.245375692178367|
[26.0,47.262,14.8...| 6.274382654241313|
[11.0,110.0,29.74...|11.905445902916304|
[17.0,101.353,26....| 10.60673224673705|
[22.0,70.367,20.5...| 8.569022381773708|
[15.0,70.367,20.5...| 8.602322828095886|
[23.0,70.367,20.5...| 8.575274527942035|
[19.0,70.367,20.5...| 8.583294001626069|
[6.0,110.23899999...| 10.90744404434513|
[10.0,110.0,29.74...| 11.8920435845878|
[28.0,46.052,14.5...| 6.191049027192053|
[18.0,70.367,20.5...| 8.588051208243524|
[17.0,70.367,20.5...| 8.592808414860977|
[11.0,86.0,21.24,...| 9.478877527552923|
[8.0,110.0,29.74,...|11.901557997822708|
[9.0,88.5,21.24,9...| 9.499567822601351|
[15.0,70.367,20.5...| 8.602322828095886|
[12.0,88.5,21.24,...| 10.30231103877464|
[20.0,70.367,20.5...| 8.578536795008615|
+--------------------+------------------+
only showing top 20 rows